In [1]:
import tensorflow as tf
import json, os

# Tested with TensorFlow 2.1.0
print('version={}, CUDA={}, GPU={}, TPU={}'.format(
    tf.__version__, tf.test.is_built_with_cuda(),
    # GPU attached?
    len(tf.config.list_physical_devices('GPU')) > 0,
    # TPU accessible? (only works on Colab)
    'COLAB_TPU_ADDR' in os.environ))

version=2.1.0, CUDA=True, GPU=True, TPU=False


In [3]:
data_path = "/storage/ECAL/training_data/window_data/electrons/recordio_v1"
models_path = "/storage/ECAL/deepcluster/models/rnn_models"

In [266]:
# This dictionary specifies what "features" we want to extract from the
# tf.train.Example protos (i.e. what they look like on disk). We only
# need the image data "img_64" and the "label". Both features are tensors
# with a fixed length.
# You need to specify the correct "shape" and "dtype" parameters for
# these features.
feature_spec = {
}

seed_features = {
                    tf.float32: ["seed_eta", "seed_phi", "seed_iz", "en_seed", "et_seed",  # "en_seed_calib", "et_seed_calib", 
                    "seed_f5_r9","seed_f5_sigmaIetaIeta", "seed_f5_sigmaIetaIphi",
                    "seed_f5_sigmaIphiIphi","seed_f5_swissCross","seed_etaWidth",
                    "seed_phiWidth","seed_nxtals"]
}

# For the moment include the seed as cluster and not the seed variables
#for typ,flist in seed_features.items():
#    for feat in flist:
#        feature_spec[feat] = tf.io.FixedLenFeature(shape=[1], dtype=typ)

clusters_features = {   tf.int64 :["is_seed", "in_scluster","cl_nxtals"], 
                     tf.float32: ["cluster_dphi","en_cluster","et_cluster",  #"en_cluster_calib", "et_cluster_calib",
                               "cl_f5_r9", "cl_f5_sigmaIetaIeta", "cl_f5_sigmaIetaIphi",
                                "cl_f5_sigmaIphiIphi","cl_f5_swissCross","cl_etaWidth",
                                "cl_phiWidth"] }

for typ,flist in clusters_features.items():
    for feat in flist:
        feature_spec[feat] = tf.io.VarLenFeature(dtype=typ)
        

def parse_example(serialized_example):
  # Convert string to tf.train.Example and then extract features/label.
    features = tf.io.parse_single_example(serialized_example, feature_spec)
    
    to_be_stacked = []
    for typ, feats in clusters_features.items():
        for f in feats:
            if f =="in_scluster":continue
            if typ == tf.int64:
                to_be_stacked.append(tf.sparse.to_dense(tf.cast(features[f], tf.float32)))
            else:
                to_be_stacked.append(features[f].values)
                
    a = tf.transpose(tf.stack(to_be_stacked))
    label = tf.expand_dims(tf.sparse.to_dense(tf.cast(features["in_scluster"],tf.float32)),1)
    
    return a, label


In [305]:
batch_size = 200
n_features = 12
steps_per_epoch = 1e6 // batch_size
eval_steps_per_epoch = 3e5 // batch_size

# Create datasets from TFRecord files.
train_ds = tf.data.TFRecordDataset(tf.io.gfile.glob(
    '{}/training-*'.format(data_path)))
train_ds = train_ds.map(parse_example)
train_ds = train_ds.padded_batch(batch_size, padded_shapes=([None,12],[None,1]),
                                  padding_values=(0.,0.),drop_remainder=True).repeat()

eval_ds = tf.data.TFRecordDataset(tf.io.gfile.glob(
    '{}/validation-*'.format(data_path)))
eval_ds = eval_ds.map(parse_example)
eval_ds = eval_ds.padded_batch(batch_size, padded_shapes=([None,12],[None,1]),
                                   padding_values=(0.,0.),drop_remainder=True).repeat()


In [287]:
from tensorflow import keras
from tensorflow.keras import layers

In [312]:
input = layers.Input(shape=[None,n_features], batch_size=batch_size)
mask = layers.Masking(mask_value=0)(input)


#norm = layers.experimental.preprocessing.Normalization()(mask)


rnn = layers.LSTM(units=50,activation="relu", return_sequences=True)(mask)

dense1 = layers.Dense(50,  activation="relu")
dense2 = layers.Dense(1,  activation="sigmoid")

output_dense1 = layers.TimeDistributed(dense1)(rnn)
output = layers.TimeDistributed(dense2)(output_dense1)


model = keras.Model(input, output, name="rnn_model1")

model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=keras.optimizers.Adam(), metrics=["accuracy", "AUC"])

In [310]:
model.summary()

Model: "rnn_model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        [(200, None, 12)]         0         
_________________________________________________________________
masking_26 (Masking)         (200, None, 12)           0         
_________________________________________________________________
normalization_2 (Normalizati (200, None, 12)           25        
_________________________________________________________________
lstm_9 (LSTM)                (200, None, 50)           12600     
_________________________________________________________________
time_distributed_48 (TimeDis (None, None, 50)          2550      
_________________________________________________________________
time_distributed_49 (TimeDis (None, None, 1)           51        
Total params: 15,226
Trainable params: 15,201
Non-trainable params: 25
___________________________________________________

In [313]:
history = model.fit(  train_ds,
                    validation_data = eval_ds,
                   steps_per_epoch = steps_per_epoch,
                   validation_steps = eval_steps_per_epoch,
                   epochs=5,
                   verbose=True)

Train for 5000.0 steps, validate for 1500.0 steps
Epoch 1/5
5000/5000 [==============================] - 552s 110ms/step - loss: 0.0426 - accuracy: 0.9540 - AUC: 0.9616 - val_loss: 0.0391 - val_accuracy: 0.9570 - val_AUC: 0.9685
Epoch 2/5
5000/5000 [==============================] - 550s 110ms/step - loss: 0.0385 - accuracy: 0.9573 - AUC: 0.9691 - val_loss: 0.0384 - val_accuracy: 0.9576 - val_AUC: 0.9695
Epoch 3/5
5000/5000 [==============================] - 549s 110ms/step - loss: 0.0378 - accuracy: 0.9580 - AUC: 0.9702 - val_loss: 0.0372 - val_accuracy: 0.9588 - val_AUC: 0.9715
Epoch 4/5
5000/5000 [==============================] - 551s 110ms/step - loss: 0.0367 - accuracy: 0.9591 - AUC: 0.9719 - val_loss: 0.0362 - val_accuracy: 0.9598 - val_AUC: 0.9731
Epoch 5/5
5000/5000 [==============================] - 552s 110ms/step - loss: 0.0359 - accuracy: 0.9599 - AUC: 0.9733 - val_loss: 0.0357 - val_accuracy: 0.9602 - val_AUC: 0.9743


In [314]:
history

In [316]:
model.save(models_path+"/model_v1.h5")